# Automatic algorithm/strategy selection
Here we will show you how you can perform an algorithm selection, helping you choose the best optimization algorithm for your specific problem.

In [1]:
from metaheuristic_designer import simple
from metaheuristic_designer.operators import OperatorVector
from metaheuristic_designer.selectionMethods import ParentSelection, SurvivorSelection
from metaheuristic_designer.algorithms import AlgorithmSelection, StrategySelection
from metaheuristic_designer.initializers import UniformVectorInitializer
from metaheuristic_designer.strategies import HillClimb, SA, ES, GA, DE, PSO, RandomSearch
from metaheuristic_designer.benchmarks import *
import numpy as np

For the following examples, we will use the Rastrigin function for which we need to find the minimum value. This is a very hard optimization problem and will allow us to compare the performance of different algorithms.

In [2]:
rastrigin_func = Rastrigin(size=2)

## Algorithm selection
We will first explore the AlgorithmSelection class, which launches each of the specified algorithms multiple times and gets some statistics about the evaluation.

We begin by defining the algorithm we are going to use. The ```simple``` package gives us premade implementations of well known optimization algorithms with a single function call, although customization is limited.

In [3]:
# The parameters common to all algorithms
params = {
    # Stopping condition
    "stop_cond": "neval",
    "neval": 5e4,
    # Encoding
    "encoding": "real",
    # Disable verbose, the output gets very cluttered otherwise
    "verbose": False,
    # Number of times to launch each algorithm
    "repetitions": 10,
}

# Define the algorithms we are going to use
algorithms = [
    simple.random_search(params, rastrigin_func),
    simple.hill_climb(params, rastrigin_func),
    simple.simulated_annealing(params, rastrigin_func),
    simple.evolution_strategy(params, rastrigin_func),
    simple.genetic_algorithm(params, rastrigin_func),
    simple.differential_evolution(params, rastrigin_func),
    simple.particle_swarm(params, rastrigin_func),
]

# Instanciate the AlgorithmSelection class
algorithm_search = AlgorithmSelection(algorithms, params)

# Launch all the algorithms and get the best overall solution
solution, best_fitness, report = algorithm_search.optimize()

The optimize function should have returned the best overall solution and a report with statistics about the execution

In [4]:
print(f"best solution: {solution}")
print(f"fitness: {best_fitness}")

best solution: [ 3.13488197e-09 -1.38041953e-09]
fitness: 0.0


In [5]:
report.sort_values("fitness_avg")

,name,realtime_min,realtime_avg,realtime_max,realtime_std,cputime_min,cputime_avg,cputime_max,cputime_std,fitness_min,fitness_avg,fitness_max,fitness_std
6,PSO,0.320830,0.323849,0.328765,0.002511,0.320480,0.323450,0.328321,0.002474,0.000000,0.000000,0.000000,0.000000
5,DE,0.337896,0.341669,0.348524,0.003237,0.337488,0.341250,0.348091,0.003226,0.000000,0.000000,0.000000,0.000000
0,RandomSearch,0.619846,0.625561,0.633625,0.004615,0.619027,0.624651,0.632502,0.004586,0.022594,0.077547,0.157075,0.043900
4,GA,1.075304,1.121144,1.278701,0.059770,1.073753,1.119371,1.276689,0.059719,0.364133,1.592823,2.204041,0.593722
3,ES,0.434935,0.437549,0.441390,0.002112,0.434280,0.436882,0.440736,0.002113,2.062560,5.418260,10.312534,2.285147
1,HillClimb,16.078056,16.317191,16.422970,0.105949,16.054514,16.291514,16.397337,0.105139,9.964237,26.127000,44.942332,13.457069
2,SA,16.385299,16.551283,16.685527,0.101234,16.361040,16.526428,16.660703,0.100510,8.345649,28.927913,49.747446,13.501412


With this, we can clearly see that the best algorithm for this task seems to be the Differential Evolution algorithm, and surprisingly, RandomSearch performs better than HillClimb, SA, ES and even GA. This could be caused by some misconfiguration of parameters, but since we just used the default implementation, we have no way of changing this.

## Strategy Selection
We could have specified the algorithms directly constructing them piece by piece, but unless you need to try some specific implementation of an algorithm, it is much simpler to search for the best Search Strategy. This is where the StrategySearch class is used. It is very similar to the AlgorithmSelection class, but using search strategies.

This also has the advantage that you can choose a custom name for each strategy.

In [6]:
# Initializer for sinlge-solution algorithms
single_initializer = UniformVectorInitializer(rastrigin_func.vecsize, rastrigin_func.low_lim, rastrigin_func.up_lim, pop_size=1)

# Initializer for population-based algorithms
pop_initializer = UniformVectorInitializer(rastrigin_func.vecsize, rastrigin_func.low_lim, rastrigin_func.up_lim, pop_size=100)

# Define strategies to be tested
strategies = [
    RandomSearch(pop_initializer),
    HillClimb(
        single_initializer,
        OperatorVector("RandNoise", {"distrib": "Gauss", "F": 1e-3}),
        name="HillClimb",
    ),
    SA(
        single_initializer,
        OperatorVector("RandNoise", {"distrib": "Gauss", "F": 1e-3}),
        params={"iter": 100, "temp_init": 1, "alpha": 0.995},
        name="SA",
    ),
    SA(
        pop_initializer,
        OperatorVector("RandNoise", {"distrib": "Gauss", "F": 1e-3}),
        params={"iter": 100, "temp_init": 1, "alpha": 0.995},
        name="ParallelSA",
    ),
    ES(
        pop_initializer,
        mutation_op=OperatorVector("RandNoise", {"distrib": "Gauss", "F": 5e-3}),
        survivor_sel=SurvivorSelection("(m+n)"),
        params={"offspringSize": 150},
        name="ES-(μ+λ)",
    ),
    ES(
        pop_initializer,
        mutation_op=OperatorVector("RandNoise", {"distrib": "Gauss", "F": 5e-2}),
        survivor_sel=SurvivorSelection("(m,n)"),
        params={"offspringSize": 700},
        name="ES-(μ,λ)",
    ),
    GA(
        pop_initializer,
        mutation_op=OperatorVector("RandNoise", {"distrib": "Gauss", "F": 1e-3}),
        cross_op=OperatorVector("Multipoint"),
        parent_sel=ParentSelection("Tournament", {"amount": 60, "p": 0.1}),
        survivor_sel=SurvivorSelection("Elitism", {"amount": 10}),
        params={"pcross": 0.9, "pmut": 0.1},
        name="GA",
    ),
    DE(
        pop_initializer,
        OperatorVector("DE/best/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/best/1",
    ),
    DE(
        pop_initializer,
        OperatorVector("DE/rand/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/rand/1",
    ),
    DE(
        pop_initializer,
        OperatorVector("DE/current-to-best/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/current-to-best/1",
    ),
    PSO(pop_initializer, {"w": 0.7, "c1": 1.5, "c2": 1.5}, name="PSO"),
]

# Instanciate the StrategySelection class
algorithm_search = StrategySelection(
    rastrigin_func,
    strategies,
    algorithm_params={
        "stop_cond": "neval",
        "neval": 5e4,
        "verbose": False,
    },
    params={"verbose": True, "repetitions": 10},
)

# Launch all the algorithms and get the best overall solution
solution, best_fitness, report = algorithm_search.optimize()

Running 11 algorithms 10 times each.


In [7]:
print(f"best solution: {solution}")
print(f"fitness: {best_fitness}")

best solution: [-1.15167864e-09 -2.69260103e-09]
fitness: 0.0


In [8]:
report.sort_values("fitness_avg")

,name,realtime_min,realtime_avg,realtime_max,realtime_std,cputime_min,cputime_avg,cputime_max,cputime_std,fitness_min,fitness_avg,fitness_max,fitness_std
7,DE/best/1,0.346488,0.349959,0.359481,0.004314,0.346283,0.349694,0.359061,0.004242,0.000000e+00,0.000000,0.000000,0.000000
10,PSO,0.320692,0.321376,0.322729,0.000570,0.320580,0.321276,0.322608,0.000569,0.000000e+00,0.000000,0.000000,0.000000
9,DE/current-to-best/1,0.347170,0.353106,0.360369,0.004117,0.346999,0.352843,0.360064,0.004039,0.000000e+00,0.000000,0.000000,0.000000
8,DE/rand/1,0.346479,0.351920,0.359656,0.004634,0.346317,0.351633,0.359406,0.004552,0.000000e+00,0.000000,0.000000,0.000000
0,RandomSearch,0.617534,0.622090,0.630789,0.004175,0.617069,0.621614,0.630302,0.004174,2.832917e-03,0.112076,0.271338,0.099321
5,"ES-(μ,λ)",0.334366,0.337068,0.343093,0.002496,0.334120,0.336819,0.342819,0.002487,1.020577e-05,1.193976,3.979834,1.223069
6,GA,1.478600,1.970084,2.312715,0.231368,1.476270,1.967048,2.308984,0.231000,2.173906e-11,1.591934,4.974790,1.343091
4,ES-(μ+λ),0.423671,0.426799,0.438421,0.004199,0.423324,0.426423,0.438019,0.004189,9.949593e-01,2.089413,4.974791,1.587088
3,ParallelSA,0.484181,0.487019,0.490149,0.001660,0.483726,0.486572,0.489741,0.001666,9.949668e-01,2.325069,4.039412,1.264064
1,HillClimb,16.016783,16.196935,16.459515,0.140268,15.993592,16.172001,16.433834,0.139821,9.949591e-01,15.421752,28.853554,10.594624
